### **HuggingFace Login**

In [10]:
from huggingface_hub import login
login()

### **Import Libairies**

In [10]:

from IPython.display import display

import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback



### **Read Datasets**

In [2]:
'''
df_train = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/train.csv')
df_test = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/test.csv')
'''
df_train = pd.read_csv('C:/Users/cauchepy/Datasets/ComputerVisionImages/kaggle_digitrecognizer/train.csv')
df_test = pd.read_csv('C:/Users/cauchepy/Datasets/ComputerVisionImages/kaggle_digitrecognizer/test.csv')

### **Short Analysis**

In [3]:
distinct_labels = df_train['label'].value_counts()
display(pd.DataFrame(distinct_labels))
classes = len(df_train["label"].unique())

,count
label,
1,4684
7,4401
3,4351
9,4188
2,4177
6,4137
0,4132
4,4072
8,4063


### **Preprocess Datasets**

In [4]:
y_train = np.array(df_train['label'])
X_train = np.array(df_train.drop('label', axis=1))
X_test = np.array(df_test)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [6]:
X_train_norm = X_train / 255.0
X_valid_norm = X_valid / 255.0
X_test_norm = X_test / 255.0

### **Model Assessment**

In [7]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Metric': ['F1 Score', 'Precision', 'Recall'],
        'Value': [f1, precision, recall]
    })
    
    cm_df = pd.DataFrame(cm, columns=df_train["label"].unique(), index=df_train["label"].unique())
    
    return metrics_df, cm_df

### **Modeling**

##### K Nearest Neighbors

In [8]:
knn_classifier = KNeighborsClassifier(n_neighbors=classes)
knn_classifier.fit(X_train_norm, y_train)

y_pred = knn_classifier.predict(X_valid_norm)

knn_classifier_assessement = evaluate_classifier(y_valid, y_pred)
display(knn_classifier_assessement[0])
display(knn_classifier_assessement[1])

,Metric,Value
0,F1 Score,0.962301
1,Precision,0.963080
2,Recall,0.962381


,1,0,4,7,3,5,8,9,2,6
1,817,0,1,0,0,4,4,0,0,1
0,0,934,1,0,0,0,0,2,0,0
4,9,17,789,3,1,2,0,13,0,1
7,2,6,5,831,0,14,0,5,3,4
3,0,10,0,0,783,0,1,1,0,19
5,4,5,0,10,0,720,7,0,3,10
8,5,2,0,0,1,4,815,0,0,0
9,0,10,0,0,1,0,0,862,0,7
2,9,15,2,15,3,18,3,3,732,13
6,4,1,1,3,10,3,0,15,0,801


### XGBoost

In [9]:
xgb_classifier = XGBClassifier(n_estimators=1000, max_depth=4, learning_rate=0.1,  eval_metric='logloss')
xgb_classifier.fit(X_train_norm, y_train, verbose=1)

y_pred = xgb_classifier.predict(X_valid_norm)

xgb_classifier_assessement = evaluate_classifier(y_valid, y_pred)
display(xgb_classifier_assessement[0])
display(xgb_classifier_assessement[1])

KeyboardInterrupt: 

### CatBoost

In [39]:
catboost_classifier = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1, verbose=2)
catboost_classifier.fit(X_train_norm, y_train)

y_pred = catboost_classifier.predict(X_valid_norm)

catboost_classifier_assessement = evaluate_classifier(y_valid, y_pred)
display(catboost_classifier_assessement[0])
display(catboost_classifier_assessement[1])

0:	learn: 2.0962791	total: 397ms	remaining: 6m 36s
2:	learn: 1.8310850	total: 907ms	remaining: 5m 1s
4:	learn: 1.6302713	total: 1.45s	remaining: 4m 48s
6:	learn: 1.4751555	total: 1.96s	remaining: 4m 37s
8:	learn: 1.3460221	total: 2.44s	remaining: 4m 28s
10:	learn: 1.2434894	total: 2.94s	remaining: 4m 24s
12:	learn: 1.1528075	total: 3.46s	remaining: 4m 22s
14:	learn: 1.0777297	total: 3.91s	remaining: 4m 17s
16:	learn: 1.0118651	total: 4.4s	remaining: 4m 14s
18:	learn: 0.9546359	total: 4.84s	remaining: 4m 9s
20:	learn: 0.8999357	total: 5.3s	remaining: 4m 7s
22:	learn: 0.8518069	total: 5.8s	remaining: 4m 6s
24:	learn: 0.8097365	total: 6.33s	remaining: 4m 6s
26:	learn: 0.7709663	total: 6.89s	remaining: 4m 8s
28:	learn: 0.7421569	total: 7.36s	remaining: 4m 6s
30:	learn: 0.7083525	total: 7.89s	remaining: 4m 6s
32:	learn: 0.6796631	total: 8.36s	remaining: 4m 5s
34:	learn: 0.6590664	total: 8.73s	remaining: 4m
36:	learn: 0.6324410	total: 9.23s	remaining: 4m
38:	learn: 0.6080263	total: 9.72s	rem

### MultiLayer Perceptron

In [13]:
class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_data, val_labels = self.validation_data
        y_pred_proba = self.model.predict(val_data)
        y_pred = np.argmax(y_pred_proba, axis=1)
        f1 = f1_score(val_labels, y_pred, average='weighted')
        print(f"\nEpoch {epoch+1} - F1 Score: {f1:.4f}")
        logs['f1_score'] = f1  # To store in logs if needed

# Define the model
mlp_classifier = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_norm.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

mlp_classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialize callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
f1_callback = F1ScoreCallback(validation_data=(X_valid_norm, y_valid))

# Train the model with the F1 callback
mlp_classifier.fit(
    X_train_norm, 
    y_train, 
    epochs=20, 
    batch_size=64,
    validation_data=(X_valid_norm, y_valid),
    callbacks=[early_stopping, f1_callback]
)

# Evaluate on validation set
y_pred_proba = mlp_classifier.predict(X_valid_norm)
y_pred = np.argmax(y_pred_proba, axis=1)

# Compute final F1 score
from sklearn.metrics import classification_report
print(classification_report(y_valid, y_pred))

mlp_classifier_assessment = evaluate_classifier(y_valid, np.round(y_pred))
display(mlp_classifier_assessment[0])
display(mlp_classifier_assessment[1])

Epoch 1/20
263/263 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step   accuracy: 0.7

Epoch 1 - F1 Score: 0.9426
525/525 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7372 - loss: 0.8364 - val_accuracy: 0.9427 - val_loss: 0.1934 - f1_score: 0.9426
Epoch 2/20
189/525 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9317 - loss: 0.2336

KeyboardInterrupt: 